In [6]:
import pandas as pd
import numpy as np
import os
import sys
from shapely.geometry import Polygon, Point
from collections import deque
from tqdm import tqdm

### Read the data

In [7]:
current_path = os.getcwd()
day = int(current_path.split('day')[1])

fn = 'day' + str(day) + '.txt'

file_content = open(fn).read().split('\n')

test_fn = 'day' + str(day) + 'test.txt'

test_file_content = open(test_fn).read().split('\n')

test_fn2 = 'day' + str(day) + 'test copy.txt'

test_file_content2 = open(test_fn2).read().split('\n')

test_fn3 = 'day' + str(day) + 'test copy 2.txt'

test_file_content3 = open(test_fn3).read().split('\n')

test_fn4 = 'day' + str(day) + 'test copy 3.txt'

test_file_content4 = open(test_fn4).read().split('\n')

### Part 1

In [8]:


working_file_content = file_content

def find_starting_point(working_file_content):
    for y in range(len(working_file_content)):
        for x in range(len(working_file_content[y])):
            if working_file_content[y][x] == 'S':
                return y, x

def check_up(working_file_content, y, x):
    return y - 1 >= 0 and working_file_content[y - 1][x] in '|7F'

def check_down(working_file_content, y, x):
    return y < len(working_file_content) - 1 and working_file_content[y + 1][x] in '|LJ'

def check_left(working_file_content, y, x):
    return x - 1 >= 0 and working_file_content[y][x - 1] in '-LF'

def check_right(working_file_content, y, x):
    return x < len(working_file_content[y]) - 1 and working_file_content[y][x + 1] in '-7J'

def breadth_first_search(working_file_content, distances, starting_point):
    queue = deque([(starting_point, 0)])
    
    while queue:
        (y, x), current_distance = queue.popleft()

        if distances[y][x] and current_distance > distances[y][x]:
            continue

        distances[y][x] = current_distance

        possible_directions = []

        if working_file_content[y][x] == '|':
            if check_up(working_file_content, y, x):
                possible_directions.append((y - 1, x))
            if check_down(working_file_content, y, x):
                possible_directions.append((y + 1, x))
        elif working_file_content[y][x] == '-':
            if check_left(working_file_content, y, x):
                possible_directions.append((y, x - 1))
            if check_right(working_file_content, y, x):
                possible_directions.append((y, x + 1))
        elif working_file_content[y][x] == 'L':
            if check_up(working_file_content, y, x):
                possible_directions.append((y - 1, x))
            if check_right(working_file_content, y, x):
                possible_directions.append((y, x + 1))
        elif working_file_content[y][x] == 'J':
            if check_up(working_file_content, y, x):
                possible_directions.append((y - 1, x))
            if check_left(working_file_content, y, x):
                possible_directions.append((y, x - 1))
        elif working_file_content[y][x] == '7':
            if check_down(working_file_content, y, x):
                possible_directions.append((y + 1, x))
            if check_left(working_file_content, y, x):
                possible_directions.append((y, x - 1))
        elif working_file_content[y][x] == 'F':
            if check_down(working_file_content, y, x):
                possible_directions.append((y + 1, x))
            if check_right(working_file_content, y, x):
                possible_directions.append((y, x + 1))
        elif working_file_content[y][x] == 'S':
            if check_up(working_file_content, y, x):
                possible_directions.append((y - 1, x))
            if check_down(working_file_content, y, x):
                possible_directions.append((y + 1, x))
            if check_left(working_file_content, y, x):
                possible_directions.append((y, x - 1))
            if check_right(working_file_content, y, x):
                possible_directions.append((y, x + 1))

        for direction in possible_directions:
            queue.append((direction, current_distance + 1))

def create_distance_grid(working_file_content):
    starting_point = find_starting_point(working_file_content)
    distances = np.full((len(working_file_content), len(working_file_content[0])), np.inf)
    breadth_first_search(working_file_content, distances, starting_point)
    return distances

def display_distance_grid(distances):
    for row in distances:
        for col in row:
            # if number < inf, round and print as int
            if isinstance(col, float):
                if np.isinf(col):
                    print(' . ', end='')
                else:
                    # make sure to take exactly three positions to print on
                    print('{:3d}'.format(int(col)), end='')
            else:
                print(' . ', end='')
        print()
    print()

def find_furthest_point(distances):
    max_distance = 0
    for row in distances:
        for col in row:
            if not np.isinf(col) and col > max_distance:
                max_distance = col
    return int(max_distance)

distance_grid = create_distance_grid(working_file_content)
# display_distance_grid(distance_grid)
print(find_furthest_point(distance_grid))



6786


### Part 2

In [9]:
def points_inside_polygon(closed_path):
    # IMPORTANT: these points should be ordered
    # Create a Shapely Polygon from the closed path
    polygon = Polygon(closed_path)

    # Find the bounding box of the polygon
    min_x, min_y, max_x, max_y = polygon.bounds

    # Generate all possible points within the bounding box
    possible_points = [(x, y) for x in range(int(min_x), int(max_x) + 1) for y in range(int(min_y), int(max_y) + 1)]

    # exclude points that are already in the closed path
    possible_points = [point for point in possible_points if point not in closed_path]

    # Filter points that are inside the polygon
    enclosed_points = list()
    for point in tqdm(possible_points):
        if polygon.contains(Point(point)):
            enclosed_points.append(point)

    return enclosed_points

In [10]:

def depth_first_search(working_file_content, starting_point):
    path = [starting_point]

    visited = set()
    
    while True:
        y, x = path[-1]
        visited.add((y, x))

        if working_file_content[y][x] == '|':
            if check_up(working_file_content, y, x) and (y - 1, x) not in visited:
                path.append((y - 1, x))
                continue
            if check_down(working_file_content, y, x) and (y + 1, x) not in visited:
                path.append((y + 1, x))
                continue
        elif working_file_content[y][x] == '-':
            if check_left(working_file_content, y, x) and (y, x - 1) not in visited:
                path.append((y, x - 1))
                continue
            if check_right(working_file_content, y, x) and (y, x + 1) not in visited:
                path.append((y, x + 1))
                continue
        elif working_file_content[y][x] == 'L':
            if check_up(working_file_content, y, x) and (y - 1, x) not in visited:
                path.append((y - 1, x))
                continue
            if check_right(working_file_content, y, x) and (y, x + 1) not in visited:
                path.append((y, x + 1))
                continue
        elif working_file_content[y][x] == 'J':
            if check_up(working_file_content, y, x) and (y - 1, x) not in visited:
                path.append((y - 1, x))
                continue
            if check_left(working_file_content, y, x) and (y, x - 1) not in visited:
                path.append((y, x - 1))
                continue
        elif working_file_content[y][x] == '7':
            if check_down(working_file_content, y, x) and (y + 1, x) not in visited:
                path.append((y + 1, x))
                continue
            if check_left(working_file_content, y, x) and (y, x - 1) not in visited:
                path.append((y, x - 1))
                continue
        elif working_file_content[y][x] == 'F':
            if check_down(working_file_content, y, x) and (y + 1, x) not in visited:
                path.append((y + 1, x))
                continue
            if check_right(working_file_content, y, x) and (y, x + 1) not in visited:
                path.append((y, x + 1))
                continue
        elif working_file_content[y][x] == 'S':
            if len(path) > 1:
                return path
            if check_up(working_file_content, y, x) and (y - 1, x) not in visited:
                path.append((y - 1, x))
                continue
            if check_down(working_file_content, y, x) and (y + 1, x) not in visited:
                path.append((y + 1, x))
                continue
            if check_left(working_file_content, y, x) and (y, x - 1) not in visited:
                path.append((y, x - 1))
                continue
            if check_right(working_file_content, y, x) and (y, x + 1) not in visited:
                path.append((y, x + 1))
                continue

        return path

def create_path(working_file_content):
    starting_point = find_starting_point(working_file_content)
    path = depth_first_search(working_file_content, starting_point)
    return path

working_file_content = file_content
path = create_path(working_file_content)
enclosed_points = points_inside_polygon(path)

print(len(enclosed_points))

100%|██████████| 3984/3984 [00:26<00:00, 152.98it/s]

495
